### Rajvardhan Deshmukh

In [95]:
import json
import requests

### Collecting Data
1 = Head
0 = Tails

In [96]:
data=[]
for i in range(30):
    res = requests.get("https://24zl01u3ff.execute-api.us-west-1.amazonaws.com/beta").text
    res = json.loads(res)
    res = res["body"][1:len(res["body"])-1].split(", ")
    res = [int(x) for x in res]
    data.append(res)

### Estimating the theta parameters - EM Algorithm

In an example like this if we have information like which coin was chosen and its respective draw then calculating bias is a very easy task. But in a scenario where we might not know which draw belongs to which coin then we need Expectation Maximization Algorithm to calculate the biases of the coins (such a scenario might occur when there is missing data/column/ etc.
There are two major steps to calculating the theta parameters in the problem statement:

#### Expectation:
At first, we will initialize the bias to random values and then use them along with the draws to calculate the probability of which coin it is using the Baye’s Formula. This probabilities let’s us determine what share of heads and tails belongs to which coin in a given draw.

#### Maximization:
Once we have the share of number of heads and tails of each coin, we maximize the theta values i.e. divide the number of heads for a coin divided by the total flips. These new theta values are then used for the next iteration of the EM Algorithm

#### Steps taken
We have to choose the initial values randomly. To ensure we have reached the global maximums for theta values I have used 4 different initial values - 0.7 and 0.6, 0.6 and 0.5, 0.7 and 0.4, 0.4 and 0.3

#### Scope
There are some research papers on choosing the initial values of the biases which affects the convergence rate majorly. There are different techniques like using PCA/supplementary information usage/Poisson Distribution Mixtures/etc


In [97]:
import numpy as np

def em(data,theta1,theta2, iterations):
    thetas = [(theta1, theta2)]
    for c in range(iterations):
        print("Iteration%d:\t%0.2f %0.2f" % (c, theta1, theta2))
        heads1, tails1, heads2, tails2 = expectation(data, theta1, theta2)
        theta1, theta2 = maximisation(heads1, tails1, heads2, tails2)
        thetas.append((theta1,theta2))    
    return thetas

def expectation(data, theta1, theta2):
    heads1, tails1 = 0,0
    heads2, tails2 = 0,0
    for draw in data:
        likelihood1 = coin_likelihood(draw, theta1)
        likelihood2 = coin_likelihood(draw, theta2)
        p1 = likelihood1 / (likelihood1 + likelihood2)
        p2 = likelihood2 / (likelihood1 + likelihood2)
        heads1 += p1 * draw.count(1)
        tails1 += p1 * draw.count(0)
        heads2 += p2 * draw.count(1)
        tails2 += p2 * draw.count(0) 
    return heads1, tails1, heads2, tails2

def maximisation(heads1, tails1, heads2, tails2):
    theta1 = heads1 / (heads1 + tails1)
    theta2 = heads2 / (heads2 + tails2)
    return theta1, theta2

def coin_likelihood(draw, bias):
    numHeads = draw.count(1)
    flips = len(draw)
    return pow(bias, numHeads) * pow(1-bias, flips-numHeads)


thetas = em(data, 0.7, 0.6, 10)
print("=======")
thetas = em(data, 0.6, 0.5, 10)
print("=======")
thetas = em(data, 0.7, 0.4, 10)
print("=======")
thetas = em(data, 0.4, 0.3, 10)
print("=======")

Iteration0:	0.70 0.60
Iteration1:	0.68 0.40
Iteration2:	0.70 0.34
Iteration3:	0.70 0.33
Iteration4:	0.70 0.33
Iteration5:	0.70 0.33
Iteration6:	0.70 0.33
Iteration7:	0.70 0.33
Iteration8:	0.70 0.33
Iteration9:	0.70 0.33
Iteration0:	0.60 0.50
Iteration1:	0.64 0.37
Iteration2:	0.69 0.33
Iteration3:	0.69 0.32
Iteration4:	0.70 0.33
Iteration5:	0.70 0.33
Iteration6:	0.70 0.33
Iteration7:	0.70 0.33
Iteration8:	0.70 0.33
Iteration9:	0.70 0.33
Iteration0:	0.70 0.40
Iteration1:	0.71 0.34
Iteration2:	0.70 0.33
Iteration3:	0.70 0.33
Iteration4:	0.70 0.33
Iteration5:	0.70 0.33
Iteration6:	0.70 0.33
Iteration7:	0.70 0.33
Iteration8:	0.70 0.33
Iteration9:	0.70 0.33
Iteration0:	0.40 0.30
Iteration1:	0.57 0.31
Iteration2:	0.65 0.31
Iteration3:	0.68 0.32
Iteration4:	0.69 0.32
Iteration5:	0.69 0.32
Iteration6:	0.70 0.33
Iteration7:	0.70 0.33
Iteration8:	0.70 0.33
Iteration9:	0.70 0.33


### Observation
We notice that there is convergence of theta values for each set of inital values to 0.7 and 0.33 which means that this is most likely the global maximum.<br>
Thus our final theta parameters for both the coins are <b> 0.7 and 0.30 <b>